### Load Business AR Data - Cohort 3

This will gather data, create the nanoid and load data into the Business AR database

## Setup

In [ ]:
%load_ext sql
%load_ext dotenv
%config SqlMagic.named_parameters="enabled" 

In [ ]:
import pandas as pd
import numpy as np
from dotenv import find_dotenv
from dotenv import load_dotenv
from nanoid import generate
load_dotenv(find_dotenv())

In [ ]:
import string
nanoid_charset = string.ascii_letters + string.digits

In [ ]:
import google.auth
credentials, project_id = google.auth.default()

In [ ]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Reset Environment Variables

In [ ]:
%dotenv -o

## Create Business Connection

In [ ]:
DB_USER = os.getenv("BUSINESS_USERNAME", "")
DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
DB_NAME = os.getenv("BUSINESS_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

business_pool = get_pool()
%sql business_pool --alias business
%sql business

In [ ]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

## Create Warehouse Connection

In [ ]:
DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
DB_NAME = os.getenv("WAREHOUSE_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

warehouse_pool = get_pool()
%sql warehouse_pool --alias warehouse
%sql warehouse

In [ ]:
%%sql
SELECT current_database(), :INSTANCE_CONNECTION_NAME as Connection, now();

## List Database Connections

In [ ]:
%sql --connections

## Query Business Database

In [ ]:
%sql business

In [ ]:
%%sql business_data <<
select * from business

In [ ]:
business_data

## Query Warehouse Database

In [ ]:
%sql warehouse

In [ ]:
%%sql colin_data <<
select n.corp_nme as legal_name
    ,c.corp_typ_cd as legal_type
    ,CASE
         WHEN c.corp_typ_cd IN('BC')
         THEN c.corp_typ_cd || c.corp_num
         ELSE c.corp_num
    END AS identifier
    ,c.bn_15 as tax_id
from colin.corporation c, colin.corp_name n
where c.corp_num = n.corp_num
  and n.end_event_id is null
  and c.corp_num in ('1161685', '1249291', '1363120', '0787872', '0910741', '1357564', '0966461')
order by c.corp_num
limit 10;


In [ ]:
colin_df = colin_data.DataFrame()
colin_df['nano_id'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
colin_df['id'] = colin_df.index + 1
colin_df

In [ ]:
%sql business
colin_df.to_sql(name='business', con=business_pool, if_exists='append', index=False)

In [ ]:
colin_df

## Load from CSV

In [ ]:
colin_df = pd.read_csv('cohort_3.csv') 
colin_df

In [ ]:
# colin_df["identifier"] = colin_df["Jurisdiction"] + colin_df['corp_num'].apply('{:0>7}'.format)
# colin_df['nano_id'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
# colin_df['tax_id'] = None
# colin_df['id'] = np.arange(1, colin_df.shape[0] + 1) + 7
# to_bar_df = pd.DataFrame()
# to_bar_df[['id','legal_name','legal_type','identifier','tax_id','nano_id']] = colin_df[['id','Name','Jurisdiction','identifier','tax_id','nano_id']]
# to_bar_df

In [ ]:
colin_df['tax_id'] = None
colin_df['ar_reminder_flag'] = None
colin_df['state'] = None
colin_df['op_state'] = None
colin_df['corp_class'] = None
to_bar_df = colin_df

In [ ]:
to_bar_df.to_sql(name='b2', con=business_pool, if_exists='append', index=False)

insert into business (select nextval('business_id_seq'::regclass), legal_name, legal_type, identifier, tax_id, email, founding_date::date, true, date_part('year',last_ar_reminder_year::date), COALESCE(NULLIF(state,''), 'ACT'), op_state, corp_class from b2 )

id
recipients
message
sent_date
token
status
business_id

In [ ]:
to_inv_email_df = pd.DataFrame()
to_inv_email_df[['id','recipients']] = colin_df[['id','email']]
to_inv_email_df['business_id'] = to_inv_email_df['id']
to_inv_email_df['token'] = colin_df.apply(lambda row: generate(nanoid_charset), axis = 1)
to_inv_email_df['message'] = colin_df.apply(lambda row: 'AR Invitation', axis = 1)
to_inv_email_df['status'] = colin_df.apply(lambda row: 'SENT', axis = 1)
to_inv_email_df

In [ ]:
to_inv_email_df.to_sql(name='inv-ch3', con=business_pool, if_exists='append', index=False)

## Output to CSV

In [ ]:
%%sql business_data <<
select b.identifier, b.legal_name, b.legal_type, i.token, i.recipients from business b, invitations i where b.id=i.business_id and b.id >= 38

In [ ]:
bdf = business_data.DataFrame()
# bdf = bdf[['legal_name','legal_type','identifier','nano_id']]
base_url='https://annualreport.business.bcregistry.gov.bc.ca/en-CA'
bdf['url'] = f'{base_url}?nanoid=' + bdf['token']
bdf

In [ ]:
import time
from datetime import datetime
from datetime import timezone

time_stamp = time.time()
now = datetime.utcfromtimestamp(time_stamp).replace(tzinfo=timezone.utc)
# local_now = now.astimezone(Pacific)
local_now = now.astimezone()
local_now.strftime("%Y.%m.%d.%H")

In [ ]:
with open('business-ar.'+local_now.strftime("%Y.%m.%d.%H")+'.csv', 'a') as f:      
    f.write('\n\n Business Annual Report\n')
    bdf.to_csv(f, sep=',', encoding='utf-8', index=False)    

## Set Invitations

In [ ]:
-- %%sql
-- insert into invitations (id, recipients, message, sent_date, token, status, additional_message, business_id)
--     select nextval('invitations_id_seq'::regclass), 'test@example.com', 'Annual Report Due', now(), nano_id, 'SENT', null, b.id 
--     from business b